In [ ]:
"""
Author: Hanshuo Wu
This file is for the visualization of results.
"""

In [ ]:
import plotly.graph_objects as go
import pye57
import numpy as np
import os

# 1. Raw data visualization

In [ ]:
folder = "to/your/path"  # Change to your local path
file = "your_file.e57"
file_path = os.path.join(folder, file)
e57 = pye57.E57(file_path)

pc = e57.read_scan_raw(0)

In [ ]:
### Downsample randomly and visualize the original data.

data = np.zeros((pc["cartesianX"].shape[0], 6))
data[:, 0] = pc["cartesianX"]
data[:, 1] = pc["cartesianY"]
data[:, 2] = pc["cartesianZ"]
data[:, 3] = pc["colorRed"]
data[:, 4] = pc["colorGreen"]
data[:, 5] = pc["colorBlue"]

#data = np.asarray(cloud.points)
print(data.shape)

n_points = data.shape[0]
idxs_sampling = np.random.choice(data.shape[0], np.min([10*(10**4), data.shape[0]]))   # Downsample parameters can be changed here if it is too sparse
data = data[idxs_sampling, :]
print(data.shape)

trace=dict(type="scatter3d",
x=data[:, 0],
y=data[:, 1],
z=data[:, 2],
mode="markers",
marker=dict(color=[f"rgb({data[i, 3]}, {data[i, 4]}, {data[i, 5]})" for i in range(data.shape[0])])
)

fig = go.Figure(layout={"width": 600, "height": 400})
fig.add_trace(trace)
fig.update_traces(marker_size = 1)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

# 2. Result Visualization

## Classification

0 - No class - White  
1 - Rustication - Red  
2 - Brick - Green  
3 - Stucco - Blue  
4 - Wood - Yellow  
5,6,7,..... - Any other classes - Black

Out of dimension points - Purple

## Segmentation on 2D data

Segmentation results on images are available in another folder for comparision.


In [12]:
color_code = np.array([[220,220,220],  # white - class 0
                       [250,0,0], # red - Rustication - class 1
                       [0,250,0], # green - Brick - class 2
                       [0,0,250], # blue - Stucco  - class 3
                       [250,250,0], # yellow - Wood  - class 4
                      ])

### Visualize the final result
scan1 = np.load("results/scan1-0829.npy")
scan2 = np.load("results/scan2-0829.npy")

# Down sampling since too big arrays are not able to visualized
idxs_sampling = np.random.choice(scan1.shape[0], np.min([10**4, scan1.shape[0]]))
scan1 = scan1[idxs_sampling, :]
scan2 = scan2[idxs_sampling, :]



result = np.concatenate((scan1,scan2),axis=0)
print(scan1.shape)
print(result.shape)
#print(result)

(10000, 7)
(10000, 7)


In [13]:
trace=dict(type="scatter3d",
x=result[:, 0],
y=result[:, 1],
z=result[:, 2],
mode="markers",
marker=dict(color=[f"rgb({color_code[int(result[i, 6])][0]}, {color_code[int(result[i, 6])][1]}, {color_code[int(result[i, 6])][2]})" for i in range(result.shape[0])])
)

fig = go.Figure(layout={"width": 600, "height": 400})
fig.add_trace(trace)
fig.update_traces(marker_size = 1)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.update_layout(
    scene=dict(
        xaxis=dict(tickmode="auto",nticks=10, autorange=True,),
        yaxis=dict(tickmode="auto",nticks=10, autorange=True,),
        zaxis=dict(tickmode="auto",nticks=10, range=[0,60],autorange=False,),
        aspectratio=dict(x=1, y=1, z=1) #ratio of XYZ axes = 1：1：1
        ))

fig.show()

In [ ]:
e57.close()

## Visulaization using open3d

In [ ]:
import open3d as o3d

In [ ]:
result_for_o3d = result
result_for_o3d[:,3:6] = np.asarray([color_code[int(i)] for i in result[:,6]])

result_pc = o3d.geometry.PointCloud()
result_pc.points = o3d.utility.Vector3dVector(result_for_o3d[:,:3])
result_pc.colors = o3d.utility.Vector3dVector(result_for_o3d[:,3:6]/255)

o3d.visualization.draw_geometries([result_pc])  #Predicted PC